In [ ]:
from textblob import TextBlob
from fuzzywuzzy import process

# Diseases with their symptoms
diseases = {
    "Cold": ['cough', 'sore throat', 'runny nose', 'fatigue'],
    "Flu": ['fever', 'body aches', 'chills', 'fatigue', 'cough', 'sore throat'],
    "COVID-19": ['fever', 'cough', 'fatigue', 'sore throat', 'loss of taste or smell', 'shortness of breath'],
    "Pneumonia": ['cough', 'fever', 'chills', 'fatigue', 'chest pain', 'shortness of breath'],
    "Tuberculosis": ['cough', 'weight loss', 'fever', 'fatigue', 'night sweats'],
    "Strep Throat": ['sore throat', 'fever', 'headache', 'swollen lymph nodes'],
    "Sinus Infection": ['nasal congestion', 'headache', 'fever', 'facial pain'],
    "Bronchitis": ['cough', 'fatigue', 'chills', 'shortness of breath', 'chest discomfort'],
    "Allergic Rhinitis": ['sneezing', 'runny nose', 'itchy eyes', 'cough'],
    "Gastroenteritis": ['nausea', 'vomiting', 'diarrhea', 'fever', 'stomach cramps'],
    "Asthma": ['wheezing','coughing','shortness of breath','chest tightness','difficulty breathing'],
    "Diabetes Mellitus": ['increased thirst', 'frequent urination', 'extreme fatigue', 'blurred vision', 'unexplained weight loss'],
    "Chickenpox": ['itchy rash', 'fever', 'fatigue', 'headache', 'loss of appetite'],
    "Migraine": ['severe headache', 'nausea', 'vomiting', 'sensitivity to light', 'sensitivity to sound'],
    "Influenza": ['fever', 'chills', 'muscle aches', 'fatigue', 'sore throat'],
    "Parkinson\'s Disease": ['tremors', 'muscle rigidity', 'bradykinesia (slow movement)', 'postural instability', 'shuffling walk'],
    "HIV/AIDS": ['fever', 'fatigue', 'swollen lymph nodes', 'night sweats', 'unexplained weight loss'],
    "Cystic Fibrosis": ['persistent cough', 'wheezing', 'shortness of breath', 'chronic lung infections', 'poor growth'],
    "Alzheimer\'s Disease": ['memory loss', 'confusion', 'difficulty performing daily tasks', 'mood swings', 'disorientation'],
    "Rheumatoid Arthritis": ['joint pain', 'swelling', 'stiffness', 'fatigue', 'fever'],
    "Stroke": ['sudden numbness', 'confusion', 'trouble speaking', 'vision problems', 'severe headache'],
    "Multiple Sclerosis": ['numbness', 'weakness', 'vision problems', 'difficulty walking', 'fatigue'],
    "Leukemia": ['fatigue', 'frequent infections', 'easy bruising', 'bone pain', 'weight loss'],
    "Lymphoma": ['swollen lymph nodes', 'fatigue', 'fever', 'night sweats', 'unexplained weight loss'],
    "Colorectal Cancer": ['blood in stool', 'changes in bowel habits', 'abdominal pain', 'fatigue', 'unexplained weight loss'],
    "Breast Cancer": ['lump in breast', 'changes in breast appearance', 'skin changes', 'pain', 'unexplained weight loss'],
    "Ovarian Cancer": ['abdominal bloating', 'pelvic pain', 'frequent urination', 'fatigue', 'unexplained weight loss'],
    "Lung Cancer": ['persistent cough', 'chest pain', 'shortness of breath', 'fatigue', 'unexplained weight loss'],
    "Kidney Disease": ['fatigue', 'swelling', 'changes in urination', 'back pain', 'high blood pressure'],
    "Renal Failure": ['swelling', 'fatigue', 'difficulty breathing', 'confusion', 'changes in urination'],
    "Kidney Stones": ['severe pain', 'blood in urine', 'nausea', 'vomiting', 'painful urination'],
    "Endometriosis": ['pelvic pain', 'heavy periods', 'pain during intercourse', 'fatigue', 'painful bowel movements'],
    "Polycystic Ovary Syndrome (PCOS)": ['irregular periods', 'excess hair growth', 'acne', 'weight gain', 'fertility issues'],
    "Menstrual Disorders": ['irregular periods', 'heavy bleeding', 'painful periods', 'absence of periods', 'mood changes'],
    "Fibromyalgia": ['widespread pain', 'fatigue', 'sleep disturbances', 'memory problems', 'headaches'],
    "Osteoporosis": ['bone fractures', 'back pain', 'loss of height', 'stooped posture', 'muscle weakness'],
    "Osteoarthritis": ['joint pain', 'stiffness', 'swelling', 'reduced range of motion', 'cracking sound in joints'],
    "Meningitis": ['fever', 'headache', 'stiff neck', 'nausea', 'sensitivity to light'],
    "Toxoplasmosis": ['fever', 'fatigue', 'muscle aches', 'headache', 'swollen lymph nodes'],
    "Necrotizing Fasciitis": ['severe pain', 'redness or swelling of the skin', 'fever', 'nausea', 'fatigue'],
    "Hepatitis A": ['fever', 'fatigue', 'loss of appetite', 'abdominal pain', 'jaundice'],
    "Hepatitis B": ['fatigue', 'nausea', 'vomiting', 'abdominal pain', 'jaundice'],
    "Hepatitis C": ['fatigue', 'loss of appetite', 'nausea', 'abdominal pain', 'jaundice'],
    "Shingles": ['painful rash', 'itching', 'burning sensation', 'fever', 'sensitivity to touch'],
    "Dengue Fever": ['high fever', 'severe headache', 'pain behind the eyes', 'joint pain', 'skin rash'],
    "Zika Virus": ['fever', 'rash', 'joint pain', 'red eyes', 'headache'],
    "West Nile Virus": ['fever', 'headache', 'muscle pain', 'rash', 'swollen lymph nodes'],
    "Rabies": ['fever', 'headache', 'nausea', 'vomiting', 'agitation'],
    "H1N1 (Swine Flu)": ['fever', 'cough', 'sore throat', 'body aches', 'fatigue'],
    "Anthrax": ['fever', 'fatigue', 'cough', 'chest pain', 'shortness of breath'],
    "Cholera": ['diarrhea', 'vomiting', 'dehydration', 'abdominal cramps', 'fever'],
    "Scarlet Fever": ['fever', 'rash', 'sore throat', 'red tongue', 'swollen lymph nodes'],
    "Whooping Cough (Pertussis)": ['severe coughing fits', 'wheezing', 'runny nose', 'fever', 'fatigue'],
    "Herpes Simplex Virus (HSV)": ['painful blisters', 'itching', 'burning sensation', 'pain during urination', 'swollen lymph nodes'],
    "Cervical Cancer": ['abnormal vaginal bleeding', 'pelvic pain', 'pain during intercourse', 'unexplained weight loss', 'fatigue'],
    "Acne": ['pimples', 'blackheads', 'whiteheads', 'cysts', 'scarring'],
    "Psoriasis": ['red, scaly patches on skin', 'itching', 'burning sensation', 'dry skin', 'thickened nails'],
    "Eczema": ['itchy skin', 'redness', 'dry patches', 'swelling', 'crusting'],
    "Anorexia Nervosa": ['extreme weight loss', 'fear of gaining weight', 'distorted body image', 'restrictive eating', 'excessive exercise'],
    "Bulimia Nervosa": ['binging', 'purging', 'severe weight fluctuations', 'obsession with body image', 'guilt after eating'],
    "Panic Disorder": ['racing heart', 'shortness of breath', 'chest pain', 'dizziness', 'feeling of choking'],
    "Generalized Anxiety Disorder": ['excessive worry', 'fatigue', 'irritability', 'difficulty concentrating', 'muscle tension'],
    "Schizophrenia": ['delusions', 'hallucinations', 'disorganized speech', 'lack of emotion', 'withdrawal from social activities'],
    "Bipolar Disorder": ['extreme mood swings', 'depression', 'mania', 'irritability', 'impulsivity'],
    "Major Depressive Disorder": ['persistent sadness', 'loss of interest', 'fatigue', 'difficulty concentrating', 'thoughts of death'],
    "Obsessive-Compulsive Disorder (OCD)": ['repetitive thoughts', 'compulsive behaviors', 'anxiety', 'difficulty functioning', 'distress from rituals'],
    "Post-Traumatic Stress Disorder (PTSD)": ['flashbacks', 'nightmares', 'irritability', 'avoidance', 'difficulty sleeping'],
    "Anemia": ['fatigue', 'pale skin', 'dizziness', 'shortness of breath', 'chest pain'],
    "Hemophilia": ['excessive bleeding', 'easy bruising', 'joint pain', 'swelling', 'nosebleeds'],
    "Blood Clots (Deep Vein Thrombosis)": ['swelling', 'pain or tenderness in leg', 'warmth or redness', 'leg cramps', 'discolored skin'],
    "Astigmatism": ['blurred vision', 'eye strain', 'headaches', 'squinting', 'difficulty seeing at night'],
    "Glaucoma": ['loss of peripheral vision', 'eye pain', 'blurred vision', 'nausea', 'vomiting'],
    "Cataracts": ['cloudy vision', 'difficulty seeing at night', 'fading colors', 'sensitivity to light', 'double vision'],
    "Retinitis Pigmentosa": ['night blindness', 'loss of peripheral vision', 'difficulty adjusting to low light', 'blurry vision', 'tunnel vision'],
    "Macular Degeneration": ['blurry vision', 'difficulty reading', 'distorted vision', 'blind spots', 'loss of central vision'],
    "Conjunctivitis (Pink Eye)": ['redness', 'itching', 'discharge', 'tearing', 'swelling'],
    "Bell\'s Palsy": ['facial drooping', 'difficulty closing eyelid', 'loss of taste on one side of tongue', 'muscle weakness', 'pain behind ear'],
    "Vertigo": ['dizziness', 'spinning sensation', 'loss of balance', 'nausea', 'vomiting'],
    "Meniere\'s Disease": ['vertigo', 'hearing loss', 'tinnitus', 'fullness in ear', 'nausea'],
    "Diabetes Insipidus": ['frequent urination', 'excessive thirst', 'fatigue', 'dry skin', 'dizziness'],
    "Graves\' Disease": ['weight loss', 'increased appetite', 'nervousness', 'rapid heartbeat', 'tremors'],
    "Hypothyroidism": ['fatigue', 'weight gain', 'sensitivity to cold', 'dry skin', 'hair loss'],
    "Hyperthyroidism": ['weight loss', 'increased appetite', 'rapid heartbeat', 'sweating', 'nervousness']
}

# Function to correct spelling
def correct_spelling(input_symptom):
    blob = TextBlob(input_symptom)
    return str(blob.correct())

# Fuzzy matching to find the closest symptom in the disease list
def find_closest_match(input_symptom, disease_symptoms):
    best_match = process.extractOne(input_symptom, disease_symptoms)
    return best_match

# Function to validate user input and ensure correct symptom entry
def get_valid_symptom(disease_symptoms):
    while True:
        symptom = input("Enter a symptom: ").strip()
        if symptom.lower() == 'done':
            return None
        # Spell correction
        corrected_symptom = correct_spelling(symptom)
        
        # Check if corrected symptom matches any in the list
        match = find_closest_match(corrected_symptom, disease_symptoms)
        if match[1] >= 80:  # Match confidence threshold (80%)
            print(f"Symptom '{symptom}' corrected to '{corrected_symptom}'")
            return corrected_symptom
        else:
            print(f"'{symptom}' is not a valid symptom. Did you mean '{match[0]}'? If so, type that.")

# Function to collect user symptoms, including validation
def collect_user_symptoms():
    user_symptoms = []
    print("\n** Enter Your Symptoms **")
    print("Type 'done' when finished.")
    
    while True:
        symptom = get_valid_symptom([symptom for disease in diseases.values() for symptom in disease])
        if symptom is None:  # End input if 'done' is typed
            break
        user_symptoms.append(symptom)

    return user_symptoms

# Function to calculate disease likelihood based on symptoms
def calculate_likelihood(diseases, user_symptoms):
    disease_scores = {}

    for disease, symptoms in diseases.items():
        matched_symptoms = sum(1 for symptom in symptoms if symptom in user_symptoms)
        
        if matched_symptoms == 0:
            disease_scores[disease] = 0
        else:
            disease_score = matched_symptoms / len(symptoms) * (len(user_symptoms) / len(symptoms)) * 100
            disease_scores[disease] = disease_score

    max_score = max(disease_scores.values(), default=1)
    disease_percentages = {disease: (score / max_score) * 100 for disease, score in disease_scores.items()}

    return disease_percentages

# Function to display disease likelihood results
def display_results(disease_percentages):
    print("\n** Disease Likelihood Based on Your Symptoms **")
    if not disease_percentages:
        print("No diseases matched the symptoms provided.")
    else:
        for disease, percentage in sorted(disease_percentages.items(), key=lambda x: x[1], reverse=True):
            print(f"{disease}: {percentage:.2f}% chance")

# Menu-driven function to interact with the user
def main():
    while True:
        print("\nMenu:")
        print("1. Add a new disease")
        print("2. Check your disease likelihood based on symptoms")
        print("3. Exit")

        choice = input("Enter your choice (1/2/3): ").strip()

        if choice == "1":
            disease_name = input("Enter the name of the disease: ").strip()
            symptoms = []
            print(f"Enter symptoms for {disease_name}. Type 'done' when finished.")
            
            while True:
                symptom = get_valid_symptom([symptom for disease in diseases.values() for symptom in disease])
                if symptom is None:
                    break
                symptoms.append(symptom)

            diseases[disease_name] = symptoms
            print(f"{disease_name} has been added with symptoms: {symptoms}")

        elif choice == "2":
            user_symptoms = collect_user_symptoms()
            disease_percentages = calculate_likelihood(diseases, user_symptoms)
            display_results(disease_percentages)

        elif choice == "3":
            print("Exiting the Symptom Checker. Goodbye!")
            break
        else:
            print("Invalid choice. Please select again.")

if __name__ == "__main__":
    main()


C:\Users\sayeem khan\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



Menu:
1. Add a new disease
2. Check your disease likelihood based on symptoms
3. Exit


Enter your choice (1/2/3):  widespread pain


Invalid choice. Please select again.

Menu:
1. Add a new disease
2. Check your disease likelihood based on symptoms
3. Exit


Enter your choice (1/2/3):  fatigue


...

Symptom 'headache' corrected to 'headache'


Enter a symptom:  swollen lymph nodes


Symptom 'swollen lymph nodes' corrected to 'swollen lymph nodes'


Enter a symptom:  done



** Disease Likelihood Based on Your Symptoms **
Strep Throat: 100.00% chance
Sinus Infection: 66.67% chance
Toxoplasmosis: 64.00% chance
West Nile Virus: 64.00% chance
Chickenpox: 42.67% chance
HIV/AIDS: 42.67% chance
Lymphoma: 42.67% chance
Meningitis: 42.67% chance
Zika Virus: 42.67% chance
Rabies: 42.67% chance
Scarlet Fever: 42.67% chance
Tuberculosis: 21.33% chance
Gastroenteritis: 21.33% chance
Influenza: 21.33% chance
Rheumatoid Arthritis: 21.33% chance
Necrotizing Fasciitis: 21.33% chance
Hepatitis A: 21.33% chance
Shingles: 21.33% chance
H1N1 (Swine Flu): 21.33% chance
Anthrax: 21.33% chance
Cholera: 21.33% chance
Whooping Cough (Pertussis): 21.33% chance
Herpes Simplex Virus (HSV): 21.33% chance
Flu: 14.81% chance
COVID-19: 14.81% chance
Pneumonia: 14.81% chance
Cold: 0.00% chance
Bronchitis: 0.00% chance
Allergic Rhinitis: 0.00% chance
Asthma: 0.00% chance
Diabetes Mellitus: 0.00% chance
Migraine: 0.00% chance
Parkinson's Disease: 0.00% chance
Cystic Fibrosis: 0.00% chance
